In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

## 依照指示取出模型特定層的資訊

In [2]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()


### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [3]:
outputs= []
def layer1_hook(module, input_, output):
    outputs.append(output)
    
model.pool.register_forward_hook(layer1_hook)

In [4]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

In [5]:
outputs

[tensor([[[[6.0406e-01, 3.4534e-01, 0.0000e+00,  ..., 0.0000e+00,
            6.8484e-02, 5.9073e-01],
           [3.9073e-01, 4.9307e-01, 3.9027e-01,  ..., 3.6572e-01,
            7.5747e-01, 1.2801e-01],
           [5.7668e-01, 2.9663e-01, 2.2913e-01,  ..., 0.0000e+00,
            1.1429e+00, 4.8236e-01],
           ...,
           [2.6287e-01, 2.7306e-01, 8.2329e-01,  ..., 3.1495e-01,
            5.1935e-01, 2.0092e-01],
           [3.9585e-01, 9.5460e-01, 4.0145e-01,  ..., 6.2864e-01,
            6.3798e-01, 0.0000e+00],
           [7.1081e-01, 4.9360e-01, 3.3785e-01,  ..., 6.6887e-01,
            3.8265e-01, 1.3243e-01]],
 
          [[3.8981e-02, 7.1436e-01, 1.8299e-01,  ..., 5.0728e-01,
            1.0108e+00, 8.6252e-01],
           [4.4648e-01, 6.6725e-01, 3.3249e-01,  ..., 8.0908e-01,
            4.8514e-01, 8.4741e-02],
           [1.1110e-01, 0.0000e+00, 2.6666e-01,  ..., 9.0766e-01,
            9.7305e-01, 5.0702e-01],
           ...,
           [7.0732e-01, 3.8981e-01, 2.

In [6]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [7]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
            m.bias.data.fill_(1)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [8]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


